In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from datetime import datetime, timedelta, date
import time
import os
import sys
import json
sys.path.append(os.path.abspath(os.path.join('..')))

/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
%load_ext autoreload
%autoreload 2

from helpers.db_query import *

# Weekly video count

In [3]:
video_dates = pd.read_csv('../data/lin_alg_moodle/videos.csv', index_col=0)
video_dates['Due_date'] = pd.to_datetime(video_dates.Due_date)

In [4]:
def select_by_year(video_dates, year):
    return video_dates.loc[video_dates.Due_date.dt.year == year]

video_dates_2017, video_dates_2018, video_dates_2019 = [select_by_year(video_dates, year) for year in [2017,2018, 2019]]

In [5]:
def weekly_count(video_dates):
    return video_dates.groupby(pd.Grouper(key='Due_date',freq='W-THU')).size().values

count_2017, count_2018, count_2019 = [weekly_count(dates) for dates in [video_dates_2017, video_dates_2018,video_dates_2019]]
print("2017", count_2017)
print("2018", count_2018)
print("2019", count_2019)

2017 [ 6  9  6 12  7]
2018 [ 7 10 10  6 13  7  8  7  9  6]
2019 [ 5  4  8  7  7 10 11 13  7  8  6  9  6]


# Features
## Video views

In [6]:
events = getVideoEvents(mode='all')

In [62]:
events.sample()

,DataPackageID,AccountUserID,VideoID,TimeStamp,EventType,SeekType,OldTime,CurrentTime,NewTime,OldSpeed,NewSpeed,Date,Year
782231,EPFL-AlgebreLineaire-2018,47670,b3b3943d538b4282b1608fbe3a8619d8,1542143265,Video.Pause,None,NaN,36.8866,NaN,NaN,NaN,2018-11-13 21:07:45,2018


In [156]:
user_events = events.loc[events.AccountUserID == '47670']

### Total views

In [26]:
def total_views(df):
    """ 
    Counts the total of videos views (rewatch included)
    Assumption: consider that a video is watched at most once per day
    """
    copy = df.copy()
    copy['Day'] = df.Date.dt.date
    #From the assumption the video view is a unique pair (video id, day)
    return len(copy.drop_duplicates(subset=['VideoID','Day'])) 
print("Total number of videos watched",total_views(user_events))

Total number of videos watched 100


### Weekly proportion (watched/replayed/interrupted)

In [51]:
def week_video_total(year):
    """
    Returns a Series with week numbers as index and the number of videos to watch per week
    """
    with open('../config/linear_algebra.json') as f:
        config = json.load(f)
    year = str(year)
    weekly_count = config[year]["WeeklyVideoCount"]
    flipped_weeks = len(config[year]["FlippedWeeks"])
    start_week = int(datetime.strptime(config[year]["StartFlipped"], '%Y-%m-%d').strftime("%V")) #Get the 1st week number
    weeks = list(range(start_week, start_week + flipped_weeks))
    return pd.DataFrame(index=weeks, data=weekly_count, columns=["Total"])

def get_dated_videos():
    PATH = '../data/lin_alg_moodle/video_with_durations.csv'
    dated_videos = pd.read_csv(PATH, index_col=0)
    dated_videos['Due_date'] = pd.to_datetime(dated_videos['Due_date']) #Convert String to datetime
    dated_videos['Year'] = dated_videos.Due_date.dt.year #Add year column
    return dated_videos

def videos_watched_on_right_week(user_events):
    dated_videos = get_dated_videos()
    first_views = user_events.merge(dated_videos, on=['VideoID', 'Year'])
    first_views['From_date'] = first_views.Due_date - timedelta(weeks=1)
    return first_views[(first_views.Date >= first_views.From_date) & (first_views.Date <= first_views.Due_date)]

def weekly_prop(user_events):
    """Compute the ratio of video events in the dataframe over the videos assigned weekly the user_events
    may only contained only the first viewings, only rewatched videos or only interrupted videos."""
    first_views = videos_watched_on_right_week(user_events)
    #Freq Weekly starting on Thursday since the last due date is on Thursday
    weekly_count = first_views.groupby(pd.Grouper(key="Date", freq="W-THU")).size().to_frame(name="Count")
    #Convert dates to week number
    weekly_count.index = [int(week) for week in weekly_count.index.strftime("%V")]
    #Number of assigned videos per week
    weekly_total = week_video_total(user_events.Year.iloc[0])
    #Merge and compute the ratio of watched
    weekly_prop = weekly_total.merge(weekly_count, left_index=True, right_index=True)
    return np.clip((weekly_prop.Count / weekly_prop.Total).values,0,1)

In [28]:
# Average and SD of the proportion of videos watched per week
def weekly_prop_watched(user_events):
    """Compute the proportion of videos watched (nb of videos watched / nb of videos assigned)"""
    first_views = user_events.drop_duplicates(subset=["VideoID"]) #Only keep the first views per video
    return weekly_prop(first_views)

def avg_weekly_prop_watched(df): 
    return weekly_prop_watched(df).mean()

def std_weekly_prop_watched(df):
    return weekly_prop_watched(df).std()

print("Videos watched")
print("Average",avg_weekly_prop_watched(user_events))
print("SG",std_weekly_prop_watched(user_events))

Videos watched
Average 0.5223901098901099
SG 0.2542535527448914


In [29]:
# Average and SD of the proportion of videos replayed per week
def weekly_prop_replayed(user_events):
    """Compute the proportion of videos replayed (nb of videos replayed / nb of videos assigned)"""
    # We assume that a student watches a video at most once per day (cannot have multiple replays in one day)
    replayed_events = user_events.copy()
    replayed_events['Day'] = replayed_events.Date.dt.date #Create column with the date but not the time
    replayed_events.drop_duplicates(subset=['VideoID', 'Day'], inplace=True) #Only keep on event per video per day
    replayed_events = replayed_events[replayed_events.duplicated(subset=['VideoID'])] # Keep the replayed videos
    return weekly_prop(replayed_events)

def avg_weekly_prop_replayed(df): 
    return weekly_prop_replayed(df).mean()

def std_weekly_prop_replayed(df):
    return weekly_prop_replayed(df).std()

print("Videos replayed")
print("Average",avg_weekly_prop_replayed(user_events))
print("SG",std_weekly_prop_replayed(user_events))

Videos replayed
Average 0.08535714285714285
SG 0.13283073854360974


#### Interruption exploration

There is no interruption event. In fact there exists an event called `Video.Stop` but it consists of the union of event types causing an interruption of the video, for instance `Video.Pause` or `Video.Load`. However I noticed that some events occurred at the same time as the end of the video: **is there a "closing event" that always occur at the end of the video?**

In order to test that, the `interruption` function returns the video events occurring in the `time_threshold` last seconds. By computing the proportion of videos where an event occurred in the last seconds and the total number of unique videos watched, we can have a approximative idea of how often an "interruption event" occurs.

In [80]:
def interruption(df, time_threshold=60):
    """Returns events occurring in the last time_threshold seconds (max one event per video) """
    user_event['Day'] = user_event.Date.dt.date #Create column with the date but not the time
    dated_videos = get_dated_videos()
    user_event = user_event.merge(dated_videos, on=['VideoID', 'Year'])
    return user_event[abs(user_event.CurrentTime - user_event.Duration) < time_threshold]\
                                                                    .drop_duplicates(subset=['VideoID'])

print("Number of videos containing an closing event:",len(interruption(user_events)))
print("Total number of distinct videos watched:", len(user_events.drop_duplicates(subset=['VideoID'])))

Number of videos containing an closing event: 70
Total number of distinct videos watched: 85


In [78]:
from tqdm import tqdm

def compute_closing_prop():
    """Compute the proportion of videos containing an closing event for each user"""
    till_end_prop = []
    for userId in tqdm(events.AccountUserID.unique()):
        user_events = events.loc[events.AccountUserID == userId]
        total = len(user_events.drop_duplicates(subset=['VideoID']))
        if total != 0:
            till_end_prop.append(len(interruption(user_events)) / total)
    return np.array(till_end_prop)

till_end_prop = compute_closing_prop()

100%|██████████| 407/407 [03:18<00:00,  2.05it/s]


In [79]:
print("Average proportion of videos with closing event:",till_end_prop.mean())
print("SD of the proportion of videos with closing event:",till_end_prop.std())

Average proportion of videos with closing event: 0.5847724703777055
SD of the proportion of videos with closing event: 0.24759822134344328


We can see that in average 60% ($\pm$ 25%) of the videos end with an event occurring in the last minute.

## Actions

In [81]:
ACTIONS = ['Video.Play', 'Video.Pause', 'Video.SeekBackward', 'Video.SeekForward', 'Video.SpeedChange', 'Video.Stop']

### Total number of actions

In [97]:
def total_actions(user_events):
    """Counts the total number of actions performed across every videos"""
    return len(user_events)

print("Total number of actions:", total_actions(user_events))

Total number of actions: 2739


### Frequency of all actions

In [120]:
def frequency_all_actions(user_events):
    """Compute the frequency of actions performed per hour spent watching videos"""
    user_events = user_events.copy()
    user_events.loc[:,'Day'] = user_events.loc[:,'Date'].dt.date #Create column with the date but not the time
    user_events.drop_duplicates(subset=['VideoID', 'Day'], inplace=True) #Only keep on event per video per day
    durations = get_dated_videos()
    user_events = user_events.merge(durations, on = ["VideoID", "Year"])
    watching_time = user_events.Duration.sum() / 3600 # hours
    return total_actions(user_events) / watching_time if watching_time != 0 else 0

print(frequency_all_actions(user_events),"actions / hour")

4.72589533563976 actions / hour


### Frequency of each action

In [132]:
def count_actions(user_events, action):
    """Count the total number of events with type `action`"""
    if 'Backward' in action:
        user_events = user_events[(user_events.EventType == 'Video.Seek') & (user_events.OldTime < user_events.NewTime)]
    elif 'Forward' in action:
        user_events = user_events[(user_events.EventType == 'Video.Seek') & (user_events.OldTime > user_events.NewTime)]
    else:
        user_events = user_events[user_events.EventType == action]        
    return len(user_events)

def freq_play(user_events):
    return count_actions(user_events,'Video.Play') / total_actions(user_events)

def freq_pause(user_events):
    return count_actions(user_events,'Video.Pause') / total_actions(user_events)

def freq_seek_backward(user_events):
    return count_actions(user_events,'Video.SeekBackward') / total_actions(user_events)

def freq_seek_forward(user_events):
    return count_actions(user_events,'Video.SeekForward') / total_actions(user_events)

def freq_speed_change(user_events):
    return count_actions(user_events,'Video.SpeedChange') / total_actions(user_events)

def freq_stop(user_events):
    return count_actions(user_events,'Video.Stop') / total_actions(user_events)

total_sum = 0
for action in [freq_play,freq_pause, freq_seek_backward, freq_seek_forward, freq_speed_change, freq_stop]:
    freq = action(user_events)
    total_sum += freq
    print(action.__name__[5:], freq)
    
print("Total proportition:", total_sum)


play 0.41509433962264153
pause 0.27358490566037735
seek_backward 0.009433962264150943
seek_forward 0.018867924528301886
speed_change 0.0
stop 0.05660377358490566
Total proportition: 0.7735849056603773


Missing events: `Video.Transcript.Translate.EN`, `Video.Load`

### Pause durations

In [168]:
def pause_duration(user_events, max_duration=600):
    """Compute the time interval between each pause event and the next play event`
    Only pause durataions smaller than `max_duration` are taken into account."""
    
    pause_events = user_events[user_events.EventType.isin(["Video.Pause", "Video.Play"])].copy()
    pause_events = pause_events.sort_values(by="TimeStamp")
    pause_events['PrevEvent'] = pause_events['EventType'].shift(1)
    pause_events['Diff'] = pause_events.TimeStamp.diff().dropna()
    pause_events = pause_events[pause_events.PrevEvent == 'Video.Pause']
    pause_events = pause_events[pause_events.Diff < max_duration]
    return pause_events.Diff.values

def avg_pause_duration(user_events):
    return pause_duration(user_events).mean()

def std_pause_duration(user_events):
    return pause_duration(user_events).std()

print("Pause durations")
print("Average",avg_pause_duration(user_events))
print("SG",std_pause_duration(user_events))

Pause durations
Average 80.45820433436532
SG 118.52219796737265
